<a href="https://colab.research.google.com/github/SinaQP/Codex-Creare/blob/main/Codex_Creare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The Book of Creation**

In [29]:
%pip install openai langchain langchain-community langchain-openai faiss-cpu --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 36.8 MB/s eta 0:00:00


In [65]:
import os
import openai
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

class AvalaiClient:
    def __init__(self, api_key="", base_url="", model_name="gpt-4o-mini"):
        self.api_key, self.base_url = self._set_api_config(api_key, base_url)
        self.model_name = model_name
        self.client = openai.OpenAI(api_key=self.api_key, base_url=self.base_url)

    def get_model(self):
        return ChatOpenAI(model_name=self.model_name, openai_api_key=self.api_key, openai_api_base=self.base_url)

    def get_embeddings(self):
        return OpenAIEmbeddings(openai_api_key=self.api_key, openai_api_base=self.base_url)

    def _set_api_config(self, api_key, base_url):
        default_api_key = ""
        default_base_url = "https://api.avalai.ir/v1"
        return api_key or default_api_key, base_url or default_base_url



In [156]:
class BaseFAISSDatabase:
    def __init__(self, embeddings, db_path="faiss_index"):
        self.db_path = db_path
        self.embeddings = embeddings
        self.db = self._load_or_initialize_db()

    def _load_or_initialize_db(self):
        if os.path.exists(self.db_path):
            db = FAISS.load_local(self.db_path, self.embeddings, allow_dangerous_deserialization=True)
            if db.index.ntotal > 0:
                return db
        return FAISS.from_texts(["PLACEHOLDER"], self.embeddings)  # Ensure at least one entry

    def add_texts(self, texts):
        unique_texts = list(set(texts))
        if unique_texts:  # Prevent adding empty lists
            self.db.add_texts(unique_texts)
            self.db.save_local(self.db_path)

    def search(self, query, k=5):
        return [n.page_content for n in self.db.similarity_search(query, k=k)]

    def clear(self):
        self.db = FAISS.from_texts(["PLACEHOLDER"], self.embeddings)
        self.db.save_local(self.db_path)

    def remove_duplicates(self):
        all_data = self.view_all_data()
        if not all_data:  # Prevent errors when DB is empty
            return
        unique_texts = list(set(all_data))
        self.db = FAISS.from_texts(unique_texts, self.embeddings)
        self.db.save_local(self.db_path)

    def view_all_data(self):
        return [n.page_content for n in self.db.similarity_search("PLACEHOLDER", k=1000) if n.page_content != "PLACEHOLDER"]


In [159]:
avalai_client = AvalaiClient()
llm = avalai_client.get_model()
embeddings = avalai_client.get_embeddings()
db = CoolNameDatabase(embeddings)


In [157]:
class CoolNameDatabase(BaseFAISSDatabase):
    def add_names(self, names):
        self.add_texts(names)

    def search_similar_names(self, theme, k=5):
        return self.search(theme, k=k)

In [167]:
from langchain.vectorstores import FAISS
past_names = [
    "Tempus", "Cinematic Harmony", "TSE Visualizer", "Soul Eater",
    "Old-Cleaner", "Future Gadget Lab", "Excel-Alchemist", "Athina", "Codex Crearer",
]

db.add_names(past_names)

In [125]:
from langchain.prompts import PromptTemplate

def generate_cool_names(theme: str = "futuristic", num_names: int = 10):
    """Generates multiple cool names based on past names and a given theme."""
    context = "\n".join(db.search_similar_names(theme))

    prompt = PromptTemplate.from_template(
        """
        Based on the following cool names: {context},
        generate {num_names} unique and cool names that fit the theme: {theme}.
        Return only the names, separated by commas.
        """
    )
    print(context)

    response = llm.predict(prompt.format(context=context, theme=theme, num_names=num_names))
    return [name.strip() for name in response.split(",")]

In [127]:
theme = "SEO Blog Articles"
cool_names = generate_cool_names(theme, 5)

Excel-Alchemist
Excel-Alchemist
PLACEHOLDER
s
e


In [128]:
print(cool_names)

['SEO-Sorcerer', 'Keyword-Kingpin', 'Rank-Renegade', 'Content-Crafter', 'Traffic-Tactician']


In [169]:
db.add_names(["Rank-Raiser"])

In [171]:
db.remove_duplicates()
print("Database Contents:", db.view_all_data())

Database Contents: ['Rank-Raiser', 'Excel-Alchemist', 'Old-Cleaner', 'Future Gadget Lab', 'Tempus', 'Codex Crearer', 'Athina', 'Soul Eater', 'Cinematic Harmony', 'TSE Visualizer']
